In [1]:
# Лабораторная работа 3 по дисциплине МРЗвИС
# Выполнена студентом группы 121702
# БГУИР Заломов Роман Андреевич
#
# Вариант 15: Реализовать модель рекуррентной сети с цепью нейросетевых моделей управляемых рекуррентных блоков 
# с логарифмической функцией активации (гиперболический арксинус) выходного сигнала на скрытом слое
#
# 24.12.24
# Данный файл содержит код построения графиков различных зависимостей

In [2]:
import numpy as np
from gru import (
    GRUModel,
    fibonacci_generator,
    one_by_n_generator,
    plus_one_minus_one_generator,
    squared_generator,
    half_generator,
    arithmetic_progression,
    create_sliding_window_batches,
    one_half_generator,
    sin_generator,    
    mape
)
from gru_with_adam import GRUAdam
import plotly.graph_objects as go
from math import pi

In [3]:
def scale_sequence(sequence):
    mean = np.mean(sequence)
    deviation = np.std(sequence)    
    scaled = (np.array(sequence) - mean) / deviation    
    return scaled, mean, deviation

def descale_sequence(scaled, mean, deviation):
    descaled = scaled * deviation + mean
    return descaled

def predict_next_n_elements(sequence, window_size, batch_size, hidden_size, n,
                            max_epochs, verbosity, lr, use_adam: bool = False):
    scaled, mean, deviation = scale_sequence(sequence)        
    # scaled = sequence  
    X, y = create_sliding_window_batches(scaled, window_size, batch_size, n)
    if not use_adam:
        model = GRUModel(window_size, hidden_size, n)
    else:
        model = GRUAdam(window_size, hidden_size, n)
    model.train(X, y, lr, max_epochs, verbosity=verbosity)
    train_pred, _, _ = model.forward(X)
    training_mape = mape(y, train_pred)           
    x_last = np.vstack(tuple(scaled[-window_size:] for _ in range(batch_size)))
    x_last = np.expand_dims(x_last, axis=0)    
    pred, _, _ = model.forward(x_last)
    descaled_pred = descale_sequence(pred[-1][-1], mean, deviation)
    # descaled_pred = pred[-1][-1]
    return training_mape, model, descaled_pred, mean, deviation

In [17]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
epochs = list(range(100, 2100, 100))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_epochs: list[int] = []

seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 20
output_size = 3

verbosity = 100000
lr = 1e-2
use_adam = False

sequence = list(sin_generator(seq_length + output_size, alpha=10, beta=pi/4, omega=pi/6))

for max_epochs in epochs:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)
            

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.413583
MAPE: 0.875836
TRAINING FINISHED
Epoch 100/100, Loss: 0.412435
MAPE: 0.878887
TRAINING FINISHED
Epoch 100/100, Loss: 0.402150
MAPE: 0.866519
TRAINING FINISHED
Epoch 100/100, Loss: 0.396965
MAPE: 0.834600
TRAINING FINISHED
Epoch 100/100, Loss: 0.408666
MAPE: 0.840113
TRAINING FINISHED
Epoch 200/200, Loss: 0.329652
MAPE: 0.748664
TRAINING FINISHED
Epoch 200/200, Loss: 0.316063
MAPE: 0.706690
TRAINING FINISHED
Epoch 200/200, Loss: 0.294389
MAPE: 0.707296
TRAINING FINISHED
Epoch 200/200, Loss: 0.322457
MAPE: 0.782993
TRAINING FINISHED
Epoch 200/200, Loss: 0.289987
MAPE: 0.714173
TRAINING FINISHED
Epoch 300/300, Loss: 0.296166
MAPE: 0.776401
TRAINING FINISHED
Epoch 300/300, Loss: 0.216418
MAPE: 0.593974
TRAINING FINISHED
Epoch 300/300, Loss: 0.246362
MAPE: 0.634782
TRAINING FINISHED
Epoch 300/300, Loss: 0.207311
MAPE: 0.582617
TRAINING FINISHED
Epoch 300/300, Loss: 0.298528
MAPE: 0.702845
TRAINING FINISHED
Epoch 400/400, Loss: 0.182232
MAPE: 0

In [5]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_epochs, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=epochs, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от количества итераций обучения',
    font=dict(size=20),
    xaxis=dict(title='Количество итераций обучения'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [6]:
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()
hidden_sizes = list(range(2, 22, 2))

# np.random.seed(12345)

all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 20
window_size = 3
batch_size = 2
output_size = 3

verbosity = 100000
lr = 5e-3
max_epochs = 500
use_adam = False

sequence = list(sin_generator(seq_length + output_size, alpha=10, beta=pi/4, omega=pi/6))

for size in hidden_sizes:    

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)        
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)

        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 500/500, Loss: 0.343639
MAPE: 0.770945
TRAINING FINISHED
Epoch 500/500, Loss: 0.367624
MAPE: 0.843546
TRAINING FINISHED
Epoch 500/500, Loss: 0.417537
MAPE: 0.882940
TRAINING FINISHED
Epoch 500/500, Loss: 0.340840
MAPE: 0.887279
TRAINING FINISHED
Epoch 500/500, Loss: 0.383233
MAPE: 0.878368
TRAINING FINISHED
Epoch 500/500, Loss: 0.283701
MAPE: 0.727022
TRAINING FINISHED
Epoch 500/500, Loss: 0.401288
MAPE: 0.888707
TRAINING FINISHED
Epoch 500/500, Loss: 0.340528
MAPE: 0.886499
TRAINING FINISHED
Epoch 500/500, Loss: 0.422268
MAPE: 0.874499
TRAINING FINISHED
Epoch 500/500, Loss: 0.435586
MAPE: 0.935992
TRAINING FINISHED
Epoch 500/500, Loss: 0.366519
MAPE: 0.848996
TRAINING FINISHED
Epoch 500/500, Loss: 0.398328
MAPE: 0.910734
TRAINING FINISHED
Epoch 500/500, Loss: 0.294548
MAPE: 0.777193
TRAINING FINISHED
Epoch 500/500, Loss: 0.320095
MAPE: 0.786817
TRAINING FINISHED
Epoch 500/500, Loss: 0.318261
MAPE: 0.790957
TRAINING FINISHED
Epoch 500/500, Loss: 0.263031
MAPE: 0

In [7]:
# Plotting results for training epochs amount
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=hidden_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>количества нейронов на скрытом слое',
    font=dict(size=20),
    xaxis=dict(title='Количество нейронов на скрытом слое'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [8]:
# MAPE relation with sliding window
experiments_amount = 5
# epochs = np.linspace(1000, 35000, endpoint=True, num=10).astype(np.int32).tolist()


all_mapes = []
mapes_mean = []

all_sizes: list[int] = []

seq_length = 500
window_sizes = list(range(2, seq_length // 2, 20))
hidden_size = 10
batch_size = 2
output_size = 3

verbosity = 100000
lr = 1e-2
max_epochs = 100
use_adam = False

sequence = list(arithmetic_progression(seq_length + output_size, 1, 1))

for size in window_sizes:        

    # Training network for five times
    mapes_sum = 0    
    for _ in range(experiments_amount):
        all_sizes.append(size)              
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)    

    # Average mape for epochs amount
    mapes_mean.append(mapes_sum / experiments_amount)

TRAINING FINISHED
Epoch 100/100, Loss: 0.117221
MAPE: 0.479201
TRAINING FINISHED
Epoch 100/100, Loss: 0.146971
MAPE: 0.509149
TRAINING FINISHED
Epoch 100/100, Loss: 0.189460
MAPE: 0.614321
TRAINING FINISHED
Epoch 100/100, Loss: 0.132914
MAPE: 0.509548
TRAINING FINISHED
Epoch 100/100, Loss: 0.265346
MAPE: 0.722912
TRAINING FINISHED
Epoch 100/100, Loss: 0.013465
MAPE: 0.454479
TRAINING FINISHED
Epoch 100/100, Loss: 0.008804
MAPE: 0.380299
TRAINING FINISHED
Epoch 100/100, Loss: 0.015134
MAPE: 0.484995
TRAINING FINISHED
Epoch 100/100, Loss: 0.009315
MAPE: 0.423110
TRAINING FINISHED
Epoch 100/100, Loss: 0.021445
MAPE: 0.384293
TRAINING FINISHED
Epoch 100/100, Loss: 0.017567
MAPE: 0.814404
TRAINING FINISHED
Epoch 100/100, Loss: 0.015926
MAPE: 0.779956
TRAINING FINISHED
Epoch 100/100, Loss: 0.023994
MAPE: 0.985070
TRAINING FINISHED
Epoch 100/100, Loss: 0.008858
MAPE: 0.697760
TRAINING FINISHED
Epoch 100/100, Loss: 0.016992
MAPE: 0.797077
TRAINING FINISHED
Epoch 100/100, Loss: 0.021611
MAPE: 1

In [9]:
# Plotting results
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sizes, y=all_mapes,
               name='Все наблюдения', mode='markers')
)
fig.add_trace(
    go.Scatter(x=window_sizes, y=mapes_mean,
               name='Средние значения', mode='lines+markers')
)
fig.update_layout(
    title='Зависимость средней абсолютной ошибки от<br>размера скользящего окна',
    font=dict(size=20),
    xaxis=dict(title='Размер скользящего окна'),
    yaxis=dict(title='Средняя абсолютная ошибка'),
    width=1280,
    height=720
)
fig.show()

In [10]:
# Настройки эксперимента
experiments_amount = 5
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3
max_epochs = 2000
verbosity = 100000
lr = 1e-2

# Словарь с генераторами
sequence_generators = {
    "1, 1, 2, 3, 5, 8,...": lambda n: list(fibonacci_generator(n)),
    "1, 4, 9, 16,...": lambda n: list(squared_generator(n)),
    "1, 2, 3, 4, 5,...": lambda n: list(arithmetic_progression(n, a0=1, d=1)),    
    "1, 0.5 ,0.(3), 0.25, 0.2,...": lambda n: list(one_by_n_generator(n)),
    "1, -1, 1, -1,...": lambda n: list(plus_one_minus_one_generator(n)),
    "1, 0.5, 1, 0.5, 0.5, 1,...": lambda n: list(one_half_generator(n)),
}

all_mapes = []
mapes_mean = []
all_sequences = []

# Тренировка для каждой последовательности
for seq_name, seq_generator in sequence_generators.items():
    sequence = seq_generator(seq_length + output_size)
    print(sequence)   

    mapes_sum = 0
    for _ in range(experiments_amount):
        all_sequences.append(seq_name)
        
        train_sequence = sequence[:seq_length]
        test_sequence = sequence[-output_size:]
        training_mape, model, next_n_pred, mean, deviation = predict_next_n_elements(
            train_sequence, window_size, batch_size,
            hidden_size, output_size, max_epochs,
            verbosity, lr, use_adam=use_adam
        )
        all_epochs.append(max_epochs)
        
        # Test sequence mape
        test_sequence = sequence[-output_size:]
        test_mape = mape(test_sequence, next_n_pred)
        mapes_sum += test_mape    
        all_mapes.append(test_mape)

    mapes_mean.append(mapes_sum / experiments_amount)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.072960
MAPE: 1.098746
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.103815
MAPE: 1.317980
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.057144
MAPE: 0.943951
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.057585
MAPE: 0.985164
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.115869
MAPE: 1.297140
[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.043033
MAPE: 0.984465
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005189
MAPE: 0.295546
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.046015
MAPE: 0.983851
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.010621
MAPE: 0.430717
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.019934
MAPE: 0.698109
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005330
MAPE: 0.260070
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.008696
MAPE: 0.321637
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.005418
MAPE: 0.251922
TRAINING F

In [11]:
# Построение графика
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=all_sequences, y=all_mapes, name="Все наблюдения", mode="markers")
)
fig.add_trace(
    go.Scatter(x=list(sequence_generators.keys()), y=mapes_mean, name="Средние значения", mode="markers")
)
fig.update_layout(
    title="Зависимость средней абсолютной ошибки<br>от вида последовательности",
    font=dict(size=20),
    xaxis=dict(title="Вид последовательности"),
    yaxis=dict(title="Средняя абсолютная ошибка"),
    width=1280,
    height=720
)
fig.show()

In [12]:
# Arithmetic
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(arithmetic_progression(seq_length, 1, 1))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")  

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.028077
MAPE: 0.016677
TEST
Предсказания на тестовой выборке:
 [17.79325494 19.04303641 19.93346374]
Истинное значение на тестовой выборке:
 [18 19 20]
MAPE: 0.5693%


In [13]:
# Fibonacci
seq_length = 15
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(fibonacci_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 319.289598
MAPE: 0.616905
TEST
Предсказания на тестовой выборке:
 [116.23869509 188.07697139 304.26049756]
Истинное значение на тестовой выборке:
 [144 233 377]
MAPE: 19.2844%


In [14]:
# 1 / n
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(one_by_n_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%")

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.001076
MAPE: 0.266920
TEST
Предсказания на тестовой выборке:
 [0.06172827 0.0467618  0.04550962]
Истинное значение на тестовой выборке:
 [0.05555556 0.05263158 0.05      ]
MAPE: 10.4147%


In [15]:
# 1, -1, 1, -1, ...
seq_length = 20
window_size = 5
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(plus_one_minus_one_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 2000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

Epoch 2000/2000, Loss: 0.051418
MAPE: 0.208200
TRAINING FINISHED
Epoch 2000/2000, Loss: 0.051404
MAPE: 0.208168
TEST
Предсказания на тестовой выборке:
 [-0.16109262  0.09186779 -0.09827792]
Истинное значение на тестовой выборке:
 [-1  1 -1]
MAPE: 88.2921%


In [16]:
# n ** 2
seq_length = 10
window_size = 3
batch_size = 2
hidden_size = 10
output_size = 3

sequence = list(squared_generator(seq_length))
X, y = create_sliding_window_batches(sequence, window_size, batch_size, output_size)

verbosity = 5000
model = GRUModel(window_size, hidden_size, output_size)

# Параметры обучения
lr = 1e-3
epochs = 2000

model.train(x=X, y=y, lr=lr, max_epochs=epochs, verbosity=verbosity)

print('TEST')
y_pred, _, _ = model.forward(X)
print("Предсказания на тестовой выборке:\n", y_pred[-1][-1])
print("Истинное значение на тестовой выборке:\n", y[-1][-1])
print(f"MAPE: {mape(y[-1][-1], y_pred[-1][-1]) * 100:.4f}%") 

TRAINING FINISHED
Epoch 2000/2000, Loss: 0.452125
MAPE: 0.017158
TEST
Предсказания на тестовой выборке:
 [ 64.53483234  81.48946797 100.3979004 ]
Истинное значение на тестовой выборке:
 [ 64  81 100]
MAPE: 0.6126%
